In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, silhouette_score, adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
np.random.seed(42)

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
%ls

2015_codified_data.csv
CAD_NLP_result.csv
CD_NLP_result.csv
CHF_label.csv
Depression_NLP_result.csv
Final_emb.pth
Final_emb_1.pth
Final_emb_8_19.pth
Final_emb_8_9.pth
HF_NLP_result.csv
MS_label.csv
RA_NLP_result.csv
RA_label.csv
T1DM_NLP_result.csv
T2DM_NLP_result.csv
UC_NLP_result.csv
Untitled.ipynb
codified_patient_embed.csv
codified_patient_embed_feature_select_RA.csv
codified_patient_embed_sap.csv
codified_patient_embed_sap_feature_select_RA.csv
depress_label.csv
emb_Z0_728.csv
logReg.ipynb
logReg_updated.ipynb
logReg_updated_cur.ipynb
logReg_updated_cur_8_9-Copy1.ipynb
logReg_updated_cur_8_9.ipynb
logReg_updated_with_embeddings.ipynb
negativeRA.csv
phenotyping.ipynb
phenotyping2-Copy1_8_19.ipynb
phenotyping2_current.ipynb
phenotyping_8_10.ipynb
positiveRA.csv
selected_features.csv
selected_features_200.csv
selected_features_50.csv
selected_features_88.csv


In [4]:
# patient_embed_GAT = pd.read_csv("codified_patient_embed.csv")
# patient_embed_SAP = pd.read_csv("codified_patient_embed_sap.csv")

# patient_embed_GAT['embedding'] = patient_embed_GAT['embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))
# patient_embed_SAP['embedding'] = patient_embed_SAP['embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

In [5]:
patient_embed_GAT = pd.read_csv("codified_patient_embed_feature_select.csv")
patient_embed_SAP = pd.read_csv("codified_patient_embed_sap_feature_select.csv")

patient_embed_GAT['embedding'] = patient_embed_GAT['embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))
patient_embed_SAP['embedding'] = patient_embed_SAP['embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

FileNotFoundError: [Errno 2] No such file or directory: 'codified_patient_embed_feature_select.csv'

In [6]:
RA_label = pd.read_csv("RA_label.csv")
CHF_label = pd.read_csv("CHF_label.csv")
depression_label = pd.read_csv("depress_label.csv")
MS_label = pd.read_csv("MS_label.csv")

In [7]:
RA_label

,patient_num,RA_GoldStandard,pi
0,2,NaN,0.255266
1,3,NaN,0.137593
2,6,NaN,0.112048
3,12,NaN,0.404592
4,17,NaN,0.112048
...,...,...,...
990,10299,NaN,0.276691
991,10364,NaN,0.137593
992,10370,NaN,0.112048
993,10471,NaN,0.137593


In [6]:
RA_label

,patient_num,RA_GoldStandard,pi
0,2,NaN,0.255266
1,3,NaN,0.137593
2,6,NaN,0.112048
3,12,NaN,0.404592
4,17,NaN,0.112048
...,...,...,...
990,10299,NaN,0.276691
991,10364,NaN,0.137593
992,10370,NaN,0.112048
993,10471,NaN,0.137593


In [7]:
import pandas as pd

# Assuming you've loaded your data into a DataFrame named RA_label
unique_values = RA_label['RA_GoldStandard'].value_counts(dropna=False)

print(unique_values)

RA_GoldStandard
NaN    841
N       96
Y       48
P        8
?        2
Name: count, dtype: int64


In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def semi_supervised_kmeans_and_visualize(patient_embed_df, disease_label_df, disease_name, visualization=False):
    # Step 1: Merge and clean data
    merged_df = pd.merge(patient_embed_df, disease_label_df, on='patient_num', how='inner')
    column_name = f"{disease_name}_GoldStandard"
    merged_df = merged_df[merged_df[column_name].isin(['Y', 'N'])]
    X = np.stack(merged_df['embedding'].values)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Step 2: Randomly select 5 patients from each group to set initial centroids
    cases = merged_df[merged_df[column_name] == 'Y'].sample(5, random_state=42)
    controls = merged_df[merged_df[column_name] == 'N'].sample(5, random_state=42)
    initial_centroids = np.vstack([np.mean(controls['embedding'].tolist(), axis=0), 
                                   np.mean(cases['embedding'].tolist(), axis=0)])
    
    # Step 3: Use initial centroids to perform KMeans clustering
    kmeans = KMeans(n_clusters=2, init=initial_centroids, n_init=1, random_state=42).fit(X_scaled)
    merged_df['cluster'] = kmeans.labels_

    # Convert N -> 0 and Y -> 1
    merged_df[column_name] = merged_df[column_name].map({'N': 0, 'Y': 1})
    
    # Calculate AUC
    binary_auc = roc_auc_score(merged_df[column_name], merged_df['cluster'])
    print(f"Binary AUC for {disease_name}: {binary_auc:.4f}")
    
    distances = kmeans.transform(X_scaled)
    scores = 1 / (1 + distances[:, 1])
    distance_based_auc = roc_auc_score(merged_df[column_name], scores)
    print(f"Distance-based AUC for {disease_name}: {distance_based_auc:.4f}")
    
    # Step 4: Visualization
    if visualization:
        pca_embeddings = PCA(n_components=2).fit_transform(X_scaled)
        plt.figure(figsize=(12, 8))
        # Points by Cluster
        plt.scatter(pca_embeddings[merged_df['cluster'] == 0][:, 0], 
                    pca_embeddings[merged_df['cluster'] == 0][:, 1], 
                    c='lightgray', label='Cluster 0', alpha=0.5)
        plt.scatter(pca_embeddings[merged_df['cluster'] == 1][:, 0], 
                    pca_embeddings[merged_df['cluster'] == 1][:, 1], 
                    c='gray', label='Cluster 1', alpha=0.5)
        # Overlay true labels
        plt.scatter(pca_embeddings[merged_df[column_name] == 0][:, 0], 
                    pca_embeddings[merged_df[column_name] == 0][:, 1], 
                    c='none', edgecolor='blue', marker='o', s=100, linewidths=1.5, label=f"True Label 0 (N)")
        plt.scatter(pca_embeddings[merged_df[column_name] == 1][:, 0], 
                    pca_embeddings[merged_df[column_name] == 1][:, 1], 
                    c='none', edgecolor='red', marker='^', s=100, linewidths=1.5, label=f"True Label 1 (Y)")
        plt.title(f'PCA visualization of clusters and true labels for {disease_name}')
        plt.xlabel('PCA Dimension 1')
        plt.ylabel('PCA Dimension 2')
        plt.legend(loc='upper right')
        plt.grid(True)
        plt.show()

    return distance_based_auc


In [9]:
def case_only_kmeans_and_visualize(patient_embed_df, disease_label_df, disease_name, visualization=False):
    # Set seed for reproducibility
    np.random.seed(42)
    
    # Step 1: Merge and clean data
    merged_df = pd.merge(patient_embed_df, disease_label_df, on='patient_num', how='inner')
    column_name = f"{disease_name}_GoldStandard"
    merged_df = merged_df[merged_df[column_name].isin(['Y', 'N'])]
    X = np.stack(merged_df['embedding'].values)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Step 2: Randomly select 5 patients from the case group to set initial centroid
    cases = merged_df[merged_df[column_name] == 'Y'].sample(5)
    case_centroid = np.mean(cases['embedding'].tolist(), axis=0).reshape(1, -1)
    
    # Get the other centroid by clustering all data with just one cluster
    single_cluster_kmeans = KMeans(n_clusters=1, random_state=42).fit(X_scaled)
    other_centroid = single_cluster_kmeans.cluster_centers_
    initial_centroids = np.vstack([case_centroid, other_centroid])
    
    # Step 3: Use the initial centroids to perform KMeans clustering
    kmeans = KMeans(n_clusters=2, init=initial_centroids, n_init=1, random_state=42).fit(X_scaled)
    merged_df['cluster'] = kmeans.labels_

    # Convert N -> 0 and Y -> 1
    merged_df[column_name] = merged_df[column_name].map({'N': 0, 'Y': 1})
    
    # Calculate AUC
    binary_auc = roc_auc_score(merged_df[column_name], merged_df['cluster'])
    print(f"Binary AUC for {disease_name}: {binary_auc:.4f}")
    
    distances = kmeans.transform(X_scaled)
    scores = 1 / (1 + distances[:, 1])
    distance_based_auc = roc_auc_score(merged_df[column_name], scores)
    print(f"Distance-based AUC for {disease_name}: {distance_based_auc:.4f}")
    
    # Step 4: Visualization
    if visualization:
        pca_embeddings = PCA(n_components=2).fit_transform(X_scaled)
        plt.figure(figsize=(12, 8))
        # Points by Cluster
        plt.scatter(pca_embeddings[merged_df['cluster'] == 0][:, 0], 
                    pca_embeddings[merged_df['cluster'] == 0][:, 1], 
                    c='lightgray', label='Cluster 0', alpha=0.5)
        plt.scatter(pca_embeddings[merged_df['cluster'] == 1][:, 0], 
                    pca_embeddings[merged_df['cluster'] == 1][:, 1], 
                    c='gray', label='Cluster 1', alpha=0.5)
        # Overlay true labels
        plt.scatter(pca_embeddings[merged_df[column_name] == 0][:, 0], 
                    pca_embeddings[merged_df[column_name] == 0][:, 1], 
                    c='none', edgecolor='blue', marker='o', s=100, linewidths=1.5, label=f"True Label 0 (N)")
        plt.scatter(pca_embeddings[merged_df[column_name] == 1][:, 0], 
                    pca_embeddings[merged_df[column_name] == 1][:, 1], 
                    c='none', edgecolor='red', marker='^', s=100, linewidths=1.5, label=f"True Label 1 (Y)")
        plt.title(f'PCA visualization of clusters and true labels for {disease_name}')
        plt.xlabel('PCA Dimension 1')
        plt.ylabel('PCA Dimension 2')
        plt.legend(loc='upper right')
        plt.grid(True)
        plt.show()

    return distance_based_auc



In [10]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def kmeans_and_visualize(patient_embed_df, disease_label_df, disease_name, visualization = False):
    merged_df = pd.merge(patient_embed_df, disease_label_df, on='patient_num', how='inner')
    
    column_name = f"{disease_name}_GoldStandard"
    #removing the NaNs and other characters not 'Y' or 'N'
    merged_df = merged_df[merged_df[column_name].isin(['Y', 'N'])]

    X = np.stack(merged_df['embedding'].values)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    kmeans = KMeans(n_clusters=2, random_state=42).fit(X_scaled)
    merged_df['cluster'] = kmeans.labels_

    # N -> 0; Y -> 1
    merged_df[column_name] = merged_df[column_name].map({'N': 0, 'Y': 1})
    
    # AUC using binary cluster labels as predicted scores
    binary_auc = roc_auc_score(merged_df[column_name], merged_df['cluster'])
    print(f"Binary AUC for {disease_name}: {binary_auc:.4f}")
    
    # AUC using continuous scores based on distance to the second cluster
    distances = kmeans.transform(X_scaled)
    scores = 1 / (1 + distances[:, 1])
    distance_based_auc = roc_auc_score(merged_df[column_name], scores)
    print(f"Distance-based AUC for {disease_name}: {distance_based_auc:.4f}")


    #visualization
    if (visualization):
        pca_embeddings = PCA(n_components=2).fit_transform(X_scaled)

        plt.figure(figsize=(12, 8))

        # Points by Cluster
        plt.scatter(pca_embeddings[merged_df['cluster'] == 0][:, 0], 
                    pca_embeddings[merged_df['cluster'] == 0][:, 1], 
                    c='lightgray', label='Cluster 0', alpha=0.5)
        plt.scatter(pca_embeddings[merged_df['cluster'] == 1][:, 0], 
                    pca_embeddings[merged_df['cluster'] == 1][:, 1], 
                    c='gray', label='Cluster 1', alpha=0.5)

        # Overlay true labels
        plt.scatter(pca_embeddings[merged_df[column_name] == 0][:, 0], 
                    pca_embeddings[merged_df[column_name] == 0][:, 1], 
                    c='none', edgecolor='blue', marker='o', s=100, linewidths=1.5, label=f"True Label 0 (N)")
        plt.scatter(pca_embeddings[merged_df[column_name] == 1][:, 0], 
                    pca_embeddings[merged_df[column_name] == 1][:, 1], 
                    c='none', edgecolor='red', marker='^', s=100, linewidths=1.5, label=f"True Label 1 (Y)")

        plt.title(f'PCA visualization of clusters and true labels for {disease_name}')
        plt.xlabel('PCA Dimension 1')
        plt.ylabel('PCA Dimension 2')
        plt.legend(loc='upper right')
        plt.grid(True)
        plt.show()

        return auc



In [11]:
auc_RA_gat = semi_supervised_kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=RA_label, disease_name="RA")
print("-----------------------")
auc_RA_sap = semi_supervised_kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=RA_label, disease_name="RA")

Binary AUC for RA: 0.6927
Distance-based AUC for RA: 0.9322
-----------------------
Binary AUC for RA: 0.7655
Distance-based AUC for RA: 0.8844


In [11]:
auc_RA_gat = semi_supervised_kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=RA_label, disease_name="RA")
print("-----------------------")
auc_RA_sap = semi_supervised_kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=RA_label, disease_name="RA")

Binary AUC for RA: 0.7291
Distance-based AUC for RA: 0.9110
-----------------------
Binary AUC for RA: 0.7655
Distance-based AUC for RA: 0.8844


In [12]:
auc_RA_gat = case_only_kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=RA_label, disease_name="RA")
print("-----------------------")
auc_RA_sap = case_only_kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=RA_label, disease_name="RA")

Binary AUC for RA: 0.2709
Distance-based AUC for RA: 0.2029
-----------------------
Binary AUC for RA: 0.2345
Distance-based AUC for RA: 0.1739


In [14]:
auc_CHF_gat = semi_supervised_kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=CHF_label, disease_name="CHF")
print("-----------------------")
auc_CHF_sap = semi_supervised_kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=CHF_label, disease_name="CHF")

Binary AUC for CHF: 0.5119
Distance-based AUC for CHF: 0.5407
-----------------------
Binary AUC for CHF: 0.5179
Distance-based AUC for CHF: 0.5193


In [13]:
auc_CHF_gat = case_only_kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=CHF_label, disease_name="CHF")
print("-----------------------")
auc_CHF_sap = case_only_kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=CHF_label, disease_name="CHF")

Binary AUC for CHF: 0.5446
Distance-based AUC for CHF: 0.6424
-----------------------
Binary AUC for CHF: 0.5238
Distance-based AUC for CHF: 0.6181


In [14]:
auc_depression_gat = semi_supervised_kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=depression_label, disease_name="Depression")
print("-----------------------")
auc_depression_sap = semi_supervised_kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=depression_label, disease_name="Depression")

Binary AUC for Depression: 0.4761
Distance-based AUC for Depression: 0.5344
-----------------------
Binary AUC for Depression: 0.4882
Distance-based AUC for Depression: 0.5356


In [15]:
auc_depression_gat = case_only_kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=depression_label, disease_name="Depression")
print("-----------------------")
auc_depression_sap = case_only_kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=depression_label, disease_name="Depression")

Binary AUC for Depression: 0.4761
Distance-based AUC for Depression: 0.5344
-----------------------
Binary AUC for Depression: 0.4882
Distance-based AUC for Depression: 0.5356


In [16]:
auc_MS_gat = semi_supervised_kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=MS_label, disease_name="MS")
print("-----------------------")
auc_MS_sap = semi_supervised_kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=MS_label, disease_name="MS")

Binary AUC for MS: 0.5481
Distance-based AUC for MS: 0.6703
-----------------------
Binary AUC for MS: 0.5597
Distance-based AUC for MS: 0.6326


In [17]:
auc_MS_gat = case_only_kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=MS_label, disease_name="MS")
print("-----------------------")
auc_MS_sap = case_only_kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=MS_label, disease_name="MS")

Binary AUC for MS: 0.5481
Distance-based AUC for MS: 0.6703
-----------------------
Binary AUC for MS: 0.5597
Distance-based AUC for MS: 0.6326


In [18]:
# auc_RA_gat = kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=RA_label, disease_name="RA")
# print("-----------------------")
# auc_RA_sap = kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=RA_label, disease_name="RA")

In [12]:
# auc_CHF_gat = kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=CHF_label, disease_name="CHF")
# print("-----------------------")
# auc_CHF_sap = kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=CHF_label, disease_name="CHF")

Binary AUC for CHF: 0.4554
Distance-based AUC for CHF: 0.6235
-----------------------
Binary AUC for CHF: 0.4762
Distance-based AUC for CHF: 0.5556


In [20]:
# auc_depression_gat = kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=depression_label, disease_name="Depression")
# print("-----------------------")
# auc_depression_sap = kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=depression_label, disease_name="Depression")

Binary AUC for Depression: 0.5239
Distance-based AUC for Depression: 0.6834
-----------------------
Binary AUC for Depression: 0.4792
Distance-based AUC for Depression: 0.5445


In [14]:
# auc_MS_gat = kmeans_and_visualize(patient_embed_df=patient_embed_GAT, disease_label_df=MS_label, disease_name="MS")
# print("-----------------------")
# auc_MS_sap = kmeans_and_visualize(patient_embed_df=patient_embed_SAP, disease_label_df=MS_label, disease_name="MS")

Binary AUC for MS: 0.5243
Distance-based AUC for MS: 0.6667
-----------------------
Binary AUC for MS: 0.4636
Distance-based AUC for MS: 0.4682
